In [ ]:
#Cell 0: Imports
from pathlib import Path

import torch
import yaml
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration

In [ ]:
#Cell 1: Load bank statement structure classifier prompt
# Try the simpler version first
prompt_path = Path("/home/jovyan/nfs_share/tod/LMM_POC/prompts/bank_statement_structure_classifier_simple.yaml")

print("📄 Loading classification prompt...")
with prompt_path.open("r", encoding="utf-8") as f:
    classifier_config = yaml.safe_load(f)

print(f"✅ Loaded classifier: {classifier_config['name']}")
print(f"📋 Version: {classifier_config['version']}")
print(f"🎯 Task: {classifier_config['task']}")

In [ ]:
#Cell 2: Load Llama-3.2-Vision model
model_id = "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision-Instruct"

print("🔧 Loading Llama-3.2-Vision model...")
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

print("✅ Model loaded successfully!")

In [ ]:
#Cell 3: Load test bank statement image
image_path = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/image_003.png"

print("📷 Loading bank statement image...")
image = Image.open(image_path)
print(f"✅ Image loaded: {image.size}")
print(f"📁 Image path: {image_path}")

In [ ]:
#Cell 4: Build classification prompt from YAML
classification_instruction = classifier_config['instruction']
output_format = classifier_config['output_format']

full_prompt = f"{classification_instruction}\n\n{output_format}"

print("📝 Classification prompt constructed")
print(f"📏 Prompt length: {len(full_prompt)} characters")
print("\n" + "="*60)
print("FULL CLASSIFICATION PROMPT:")
print("="*60)
print(full_prompt)
print("="*60)

In [ ]:
#Cell 5: Generate classification response
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": full_prompt,
            },
        ],
    }
]

print("🤖 Generating classification with Llama-3.2-Vision...")

# Process the input
textInput = processor.apply_chat_template(
    messageDataStructure, add_generation_prompt=True
)
inputs = processor(image, textInput, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(**inputs, max_new_tokens=2000)
generatedOutput = processor.decode(output[0])

print("✅ Classification generated successfully!")

In [ ]:
#Cell 6: Display and classify results
print("\n" + "="*60)
print("LLAMA-3.2-VISION STRUCTURE CLASSIFICATION:")
print("="*60)
print(generatedOutput)
print("="*60)

# Extract clean response (remove chat template artifacts)
if "<|start_header_id|>assistant<|end_header_id|>" in generatedOutput:
    clean_response = generatedOutput.split("<|start_header_id|>assistant<|end_header_id|>")[1]
    clean_response = clean_response.replace("<|eot_id|>", "").strip()
    
    print("\n" + "="*60)
    print("EXTRACTED HEADERS:")
    print("="*60)
    print(clean_response)
    print("="*60)
    
    # Python-based counting and classification
    if "NO_HEADERS" in clean_response:
        column_count = 0
        structure_type = "MOBILE_APP_LIGHT_INLINE"
        headers = []
    else:
        # Look for lines containing typical header keywords
        header_keywords = ['Date', 'Transaction', 'Description', 'Amount', 'Debit', 'Credit', 'Balance', 'Particulars']
        
        header_text = None
        for line in clean_response.split('\n'):
            line = line.strip()
            # Skip empty lines
            if not line:
                continue
            # Must contain commas AND at least one header keyword
            if ',' in line and any(keyword in line for keyword in header_keywords):
                header_text = line
                break
        
        if not header_text:
            # Still not found? Take the last non-empty line (VLM might put headers at end)
            lines = [l.strip() for l in clean_response.split('\n') if l.strip()]
            if lines:
                header_text = lines[-1]
            else:
                header_text = ""
        
        # Clean up trailing periods
        header_text = header_text.rstrip('.')
        
        # Split by comma and clean
        headers = [h.strip() for h in header_text.split(',') if h.strip()]
        column_count = len(headers)
        
        # Classification based on Python count
        if column_count == 3:
            structure_type = "TABLE_3COL_SIMPLE"
        elif column_count == 4:
            structure_type = "TABLE_4COL_STANDARD"
        elif column_count == 5:
            structure_type = "TABLE_5COL_STANDARD"
        elif column_count >= 6:
            structure_type = "TABLE_EXTENDED_LOCATION"
        else:
            structure_type = "UNKNOWN"
    
    print("\n" + "="*60)
    print("PYTHON-BASED CLASSIFICATION:")
    print("="*60)
    print(f"Headers found: {headers}")
    print(f"Column count: {column_count}")
    print(f"Structure type: {structure_type}")
    print("="*60)

In [ ]:
#Cell 7: Save classification results
output_dir = Path("classification_results")
output_dir.mkdir(exist_ok=True)

# Save full output
output_path = output_dir / "llama_structure_classification.txt"
with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(generatedOutput)

# Save clean output
if "<|start_header_id|>assistant<|end_header_id|>" in generatedOutput:
    clean_output_path = output_dir / "llama_structure_classification_clean.txt"
    with clean_output_path.open("w", encoding="utf-8") as text_file:
        text_file.write(clean_response)
    print(f"✅ Clean response saved to: {clean_output_path}")

print(f"✅ Full response saved to: {output_path}")
print(f"📄 File size: {output_path.stat().st_size} bytes")

In [ ]:
#Cell 8: Test document type classifier (Mobile App vs Bank Statement)
doc_type_prompt_path = Path("/home/jovyan/nfs_share/tod/LMM_POC/prompts/document_type_classifier.yaml")

print("📄 Loading document type classifier prompt...")
with doc_type_prompt_path.open("r", encoding="utf-8") as f:
    doc_type_config = yaml.safe_load(f)

print(f"✅ Loaded classifier: {doc_type_config['name']}")
print(f"📋 Version: {doc_type_config['version']}")

# Build prompt
doc_type_instruction = doc_type_config['instruction']
doc_type_output = doc_type_config['output_format']
doc_type_prompt = f"{doc_type_instruction}\n\n{doc_type_output}"

# Generate classification
doc_type_message = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": doc_type_prompt},
        ],
    }
]

print("🤖 Classifying document type...")
doc_type_text_input = processor.apply_chat_template(
    doc_type_message, add_generation_prompt=True
)
doc_type_inputs = processor(image, doc_type_text_input, return_tensors="pt").to(model.device)
doc_type_output = model.generate(**doc_type_inputs, max_new_tokens=100)
doc_type_result = processor.decode(doc_type_output[0])

# Extract clean result
if "<|start_header_id|>assistant<|end_header_id|>" in doc_type_result:
    doc_type_clean = doc_type_result.split("<|start_header_id|>assistant<|end_header_id|>")[1]
    doc_type_clean = doc_type_clean.replace("<|eot_id|>", "").strip()
else:
    doc_type_clean = doc_type_result

print("\n" + "="*60)
print("DOCUMENT TYPE CLASSIFICATION:")
print("="*60)
print(doc_type_clean)
print("="*60)

# Determine classification
if "Mobile_APP" in doc_type_clean:
    classification = "Mobile_APP"
elif "BANK_STATEMENT" in doc_type_clean:
    classification = "BANK_STATEMENT"
else:
    classification = "UNKNOWN"

print(f"\n🏷️  Classification: {classification}")